This example is generally good but could be cleaned up: broken up inside the biggest cell, and use some more documentation.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import patches
from scipy.signal import argrelmin, argrelmax
from scipy.ndimage.filters import gaussian_filter1d as gaussian
from utils import list_all_files, ffmpeg_load_audio, ffmpeg_save_audio
from IPython.display import display, Audio
from random import shuffle
import librosa
import numpy as np

In [2]:
files = list(list_all_files('/Users/kyle/Documents/Movies/Monty Brigham', ['.mp3']))

In [3]:
len(files)

326

In [4]:
def moving_func(func, data, window_size):
    out = None
    for i in range(window_size):
        sub = data[i:i-window_size]
        if out is None:
            out = sub
        else:
            out = func([sub, out], axis=0)
    left_pad = window_size / 2
    right_pad = window_size - left_pad
    return np.pad(out, pad_width=(left_pad, right_pad), mode='edge')
def moving_min(data, window_size):
    return moving_func(np.min, data, window_size)
def moving_max(data, window_size):
    return moving_func(np.max, data, window_size)
def moving_mean(data, window_size):
    out = np.array([np.mean(data[i:i+window_size]) for i in range(0, len(data) - window_size)])
    # padding should be handled differently, this isn't a good solution
    left_pad = window_size / 2
    right_pad = window_size - left_pad
    return np.pad(out, pad_width=(left_pad, right_pad), mode='edge')

In [5]:
# need a better way of describing "pitchedness" or "harmonic power"
def noisiness(S):
    return S.std(axis=0) / S.mean(axis=0)
def maxamp(S):
    return S.max(axis=0)
def percentileamp(S, percentile):
    return np.percentile(S, percentile, axis=0)
def ampgap(S):
    return np.percentile(S, 90, axis=0) - np.percentile(S, 10, axis=0)

In [7]:
sr = 44100
skip = 5*sr
total = 30*sr
n_fft = 2048
hop_length = n_fft / 4
# shuffle(files)
percent_save = .50
i = 0
plot = False
save = True
np.seterr(divide='ignore')
for fn in files:
    print fn
    
    y, sr = ffmpeg_load_audio(fn, sr=sr)
    y = y[0,:] # take left channel. sometimes the stereo channels are out of phase
    y = y[skip:(total-skip)]
#     y.resize(total)
    y /= y.max()
    print str(int(len(y)/float(sr))) + ' seconds'
    
    if plot:
        plt.figure(figsize=(30,3))
        plt.plot(y[::100])
        plt.xlim([0, len(y)/100])
        plt.gca().xaxis.set_visible(False)
        plt.gca().yaxis.set_visible(False)
        plt.show()
    
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    logamp = librosa.logamplitude(S**2, ref_power=np.max)
#     e = ampgap(logamp)
#     e = percentileamp(logamp, 25)
#     mag, phase = librosa.magphase(S)
#     e = noisiness(mag)
#     e = librosa.feature.spectral_bandwidth(S=mag)[0]
    e = librosa.feature.rmse(S=S)[0]

    e /= e.max()
    g0 = gaussian(e, 100) # moving_max(e, 1000)
    agc = e / g0
    g1 = gaussian(agc, 25)
    g0 /= g0.max()
    g1 /= g1.max()
    agc /= agc.max()
    
    maxima = argrelmax(g1)[0]
    minima = argrelmin(g1)[0]
    minima = np.insert(minima, 0, 0)
    minima = np.append(minima, len(e)-1)
    
    if plot:
        plt.figure(figsize=(30,3))
        plt.plot(e)
        plt.plot(g0)
        plt.plot(g1)
        plt.plot(agc)

        for x in maxima: plt.axvline(x, color='green')
#         for x in minima: plt.axvline(x, color='blue')
    
    relative_peaks = []
    for peak_sample in maxima:
        found_ind = np.searchsorted(minima, peak_sample)
        left_sample = minima[found_ind-1]
        right_sample = minima[found_ind]
        peak_val = g1[peak_sample]
        left_val = g1[left_sample]
        right_val = g1[right_sample]
        relative_peaks.append([ peak_val - ((left_val + right_val) / 2), left_sample, right_sample, peak_sample ])
    
    relative_peaks.sort(reverse=True)
    total_saved = 0
    for relative_peak, left, right, peak in relative_peaks:        
        total_saved += (right - left)
        # save less than percent_saved
        if total_saved < len(g1) * percent_save:
            # highlight saved portion
            if plot:
                plt.gca().add_patch(patches.Rectangle((left, 0), (right - left), 1,
                                                     hatch='//', alpha=0.2, fill='black'))
            
            # save sample
            if save:
                sample = np.copy(y[left*hop_length:right*hop_length])
                sample /= np.abs(sample).max()
                ffmpeg_save_audio('data/birds/{}.wav'.format(i), sample, sr=sr)    
                i += 1
        else:
            break
    
    if plot:
        plt.xlim([0, len(e)])
        plt.gca().xaxis.set_visible(False)
        plt.gca().yaxis.set_visible(False)
        plt.show()

        plt.figure(figsize=(30,3))
        librosa.display.specshow(logamp, sr=sr, cmap='viridis')
        plt.tight_layout()
        plt.show()
    
#     display(Audio(data=y, rate=sr))

/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-02 Red-Throated Loon.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-03 Pacific Loon.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-04 Common Loon.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-05 Pied-Billed Grebe.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-06 Greater Shearwater.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-07 Northern Gannet.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-08 Double-Crested Cormorant.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-09 Anhinga.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 Loons to Godwits/1-10 American Bittern.mp3
20 seconds
/Users/kyle/Documents/Movies/Monty Brigham/BSoC 1-1 L